In [ ]:
import numpy
import scipy.stats
%matplotlib widget
from matplotlib import pyplot
from matplotlib.dates import num2date, date2num
import netCDF4
from ipywidgets import widgets

import sys
sys.path.append('../extern/fabm-mizer/python')
import mizer

# Function for converting from Equivalent Spherical Diameter (micrometer) to wet mass in g
def esd2mass(d): # d: equivalent spherical diameter in micrometer
    V = 4./3. * numpy.pi * (numpy.array(d) / 2e6)**3  # V: volume in m3
    return V * 1e6  # mass in g approximately equals volume in m3 multiplied by 1e6 (assumes density of 1000 kg/m3)

## NEMO-MEDUSA outputs for the Agulhas bank

* Extracted from 1/12° hindcast (1990 - 2015) on JASMIN
* 33.8 - 37 °South, 20 - 27 °East, < 200 m water depth
* Weighted depth average per grid point, weights proportional to total plankton concentration
* Then horizontally averaged

In [ ]:
# Read NetCDF
path = r'../datasets/NEMO-MEDUSA-Agulhas.nc'
with netCDF4.Dataset(path) as nc:
    print('%s contains: %s' % (path, ', '.join(nc.variables.keys())))
    temp = nc['thetao'][:, 0, 0]
    ZME = nc['ZME'][:, 0, 0]
    ZMI = nc['ZMI'][:, 0, 0]
    PHD = nc['PHD'][:, 0, 0]
    PHN = nc['PHN'][:, 0, 0]
    depth = nc['bm_int'][:, 0, 0]**2 / nc['bm2_int'][:, 0, 0]
    numtime = nc['time_counter'][:]
    time = netCDF4.num2date(numtime, nc['time_counter'].units, only_use_cftime_datetimes=False)

In [ ]:
# Plot
fig, (ax1, ax2, ax3) = pyplot.subplots(nrows=3, figsize=(12,8), sharex=True)
ax1.plot(time, temp)
ax1.grid()
ax1.set_ylabel('experienced temperature (°C)')

ax2.plot(time, depth)
ax2.grid()
ax2.set_ylabel('interaction depth (m)')

ax3.plot(time, PHN, label='non-diatom phytoplankton')
ax3.plot(time, PHD, label='diatoms')
ax3.plot(time, ZMI, label='microzooplankton')
ax3.plot(time, ZME, label='mesozooplankton')
ax3.grid()
ax3.set_ylabel('concentration (mmol N/m³)')
ax3.set_xlabel('time')
ax3.legend();

## Does the MEDUSA plankton community resemble a Sheldon size spectrum?

In [ ]:
# From ESD (um) to individual wet mass (g)
w_esd2, w_esd20, w_esd200 = esd2mass([2., 20., 200.])
print('Wet mass for cells of ESD =   2 µm: %.3g g' % w_esd2)
print('Wet mass for cells of ESD =  20 µm: %.3g g' % w_esd20)
print('Wet mass for cells of ESD = 200 µm: %.3g g' % w_esd200)

# Map MEDUSA PFTs to size classes
bins = [
    (PHN,       w_esd2,  w_esd20),
    (PHD + ZMI, w_esd20, w_esd200),
    (ZME,       1e-5,    1e-3),
]

# Key metrics per size bin
left = [l for c, l, r in bins]
width = [r - l for c, l, r in bins]
values = [c.mean() / (numpy.log10(r) - numpy.log10(l)) for c, l, r in bins]
sd = [c.std() / (numpy.log10(r) - numpy.log10(l)) for c, l, r in bins]

# Create plot
fig, ax = pyplot.subplots()
ax.bar(left, values, width=width, align='edge', ec='k')
ax.errorbar([numpy.sqrt(l*r) for c, l, r in bins], values, yerr=sd, fmt='none', ecolor='k', capsize=5)
ax.set_xscale('log')
ax.set_ylabel('concentration (mmol N/m³) per bin width in log space')
ax.set_xlabel('individual wet mass (g)')
ax.set_title('long-term average of PFT concentration per size class')
ax.grid()

## Configure the size spectrum model

In [ ]:
w_min = 1e-3
w_inf = 1e6

# Parameters of the size spectrum model (mizer, http://dx.doi.org/10.1111/2041-210X.12256)
parameters = {
    'w_min': w_min,
    'w_inf': w_inf,
    'nclass': int(100 * (numpy.log10(w_inf) - numpy.log10(w_min)) / 9.),
    'T_dependence': 1,
    'T_ref': 13.,
    'E_a': 0.63,
    'beta': 100,
    'sigma': float(numpy.log(10.)),
    'gamma': 156,
    'q': 0.82,
    #'n': 0.82, #0.93,
    'alpha': 0.2,
    'z0_type': 1,
    'z0pre': 0.1,
    'z0exp': -0.25,
    'w_s': 1000.,
    'z_s': 0.3,
    'ks': 0.,
    'SRR': 0,
    'recruitment': 0.,
    'h': 1e9, #30
    'fishing_type': 1,
    'w_minF': 1.25, # Blanchard et al 2012
    'F': 0.4 #4 #8  # note: need to put double the intended value due to fisheries equation!
}

w_esd2, w_esd20, w_esd200 = esd2mass([2., 20., 200.])

scale_factor = 10 * 0.001 * 12 * 106. / 16. # 10 g wet mass/g carbon * 0.001 g C/mg C * 12 mg C/mmol C * 106/16 mmol C/mmol N
prey = []
for long_name, w_range, ncname in [
    ('diatoms',                  (w_esd20, w_esd200), 'PHD'),
    ('non-diatom phytoplankton', (w_esd2,  w_esd20),  'PHN'),
    ('microplankton',            (w_esd20, w_esd200), 'ZMI'),
    ('mesoplankton',             (1e-5,    1e-3),     'ZME')
]:
    timeseries = mizer.datasources.TimeSeries(path, ncname, x=0, y=0, scale_factor=scale_factor, time_name='time_counter', allow_mask=True)
    prey.append(mizer.Prey(long_name, w_range, timeseries))
prey_collection = mizer.PreyCollection(*prey)
prey_collection = mizer.GriddedPreyCollection(prey_collection)

# Environmental conditions
temp = mizer.datasources.TimeSeries(path, 'thetao', x=0, y=0, time_name='time_counter', allow_mask=True)
depth = mizer.datasources.TimeSeries(path, 'bm_int**2/bm2_int', x=0, y=0, time_name='time_counter', allow_mask=True)

# create mizer model
m = mizer.Mizer(prey=prey_collection, parameters=parameters, temperature=temp, depth=depth, recruitment_from_prey=2, verbose=False)

## Time integrate

In [ ]:
# Note: the model is spun up for several years (spinup argument below).
# During this spinup it is forced with the time average of all forcing fields (no seasonal cycle!)
result = m.run(temp.times, spinup=50, verbose=True, save_spinup=False, save_loss_rates=True, save_f=True)

## Plot results

In [ ]:
normalization = 0   # 0: Sheldon spectrum (biomass per bin), 1: biomass density, 2: abundance density

# Characteristics of bins of the predator psectrum
log_bin_masses = numpy.log10(m.bin_masses)
dbin = log_bin_masses[1] - log_bin_masses[0]
bin_width = 10.**(log_bin_masses + 0.5 * dbin) - 10.**(log_bin_masses - 0.5 * dbin)

# Characteristics of bins of the prey psectrum
log_prey_masses = numpy.log10(m.prey.masses)
dprey = log_prey_masses[1] - log_prey_masses[0]
prey_bin_width = 10.**(log_prey_masses + 0.5 * dprey) - 10.**(log_prey_masses - 0.5 * dprey)

fig, ax = pyplot.subplots()

if normalization == 0:
    # Sheldon-type spectrum (expected slope = 0)
    values = result.spectrum / dbin
    prey_values = result.y[:, m.prey_indices] * result.depth[:, numpy.newaxis] / dprey
    ax.set_ylabel('wet mass (g) per log10 individual size')
elif normalization == 1:
    # biomass density (expected slope = -1)
    values = result.spectrum / bin_width
    prey_values = result.y[:, m.prey_indices] * result.depth[:, numpy.newaxis] / prey_bin_width
    ax.set_ylabel('wet mass density (g/g)')
elif normalization == 2:
    # abundance density (expected slope = -2)
    values = result.spectrum / bin_width / m.bin_masses
    prey_values = result.y[:, m.prey_indices] * result.depth[:, numpy.newaxis] / prey_bin_width / m.prey.masses
    ax.set_ylabel('abundance density (#/g)')

# Determine suitable y axis range for entire time period
minval, prey_min = values.min(), prey_values.min()
if prey_min > 0: minval = min(minval, prey_min)
maxval = max(values.max(), prey_values.max())
ax.set_ylim(minval / 10, maxval * 10)

# Plot spectrum
prey_line, = ax.loglog(m.prey.masses, prey_values[0, :], '.')
predator_line, = ax.loglog(m.bin_masses, values[0, :], '.')
ax.grid(True)
ax.set_xlabel('wet mass (g)')
title = ax.set_title(num2date(result.t[0]).strftime('%Y-%m-%d'))

# Allow for changing the time step with a slider
def update_spectrum(itime=0):
    prey_line.set_ydata(prey_values[itime, :])
    predator_line.set_ydata(values[itime, :])
    title.set_text(num2date(result.t[itime]).strftime('%Y-%m-%d'))
slider = widgets.interact(update_spectrum, itime=(0, len(result.t) - 1))

## Total biomass

In [ ]:
result.plot_biomass_timeseries()

## Landings

Remember: our fisheries parameterisation is embarassingly primitive!

In [ ]:
result.plot_timeseries('landings')
result.plot_annual_mean('landings', plot_change=True)

## The "large fish index"

The LFI describes the fraction of biomass in the fish community that is present is fish larger than "some threshold".

It is a commonly used metric to assess Good Environmental Status, or to summarize changes in the fish community under scenarios describing changes in fisheries pressure and/or climate.

Caveat: a "large fish" is typically defined in terms of length. We need to convert that to wet mass to retrieve the LFI from our results.

In [ ]:
Lmin_lfi = 40.        # minimum length (cm) for a "large fish" based on Greenstreet et al., 2011 (https://doi.org/10.1093/icesjms/fsq156)
a, b = 0.0076, 2.96   # allometric coefficients to convert length into weight (Garcia et al. subm, based on https://www.cefas.co.uk/publications/techrep/TechRep150.pdf)
wmin_lfi = a * Lmin_lfi**b
lfi = result.get_lfi_timeseries(wmin_lfi)
fig, ax = pyplot.subplots()
ax.plot_date(result.t, lfi, '-')
ax.set_ylabel('fraction of fish > %.1f g' % wmin_lfi)
ax.grid()

## Mortality rates

## Saving results

In [ ]:
# Save results in NetCDF format
result.save_as_nc('result.nc')